In [1]:
from ase.calculators.eam import EAM
from ase.io import write
from catkit.gen.adsorption import get_adsorption_sites
from mcmc import initialize_slab, spin_flip, mcmc_run, get_random_idx
from time import perf_counter
from pprint import pprint as pp

import catkit
import numpy as np

In [2]:
# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
slab = initialize_slab(Cu_alat)
potential = EAM(potential='Cu2.eam.fs')
slab.calc = potential

print(f"slab calc {slab.calc}")

# get ALL the adsorption sites
# top should have connectivity 1, bridge should be 2 and hollow more like 4
coords, connectivity, sym_idx = get_adsorption_sites(slab, symmetry_reduced=False)
print(f"In pristine slab, there are a total of {len(connectivity)} sites")

# state of each vacancy in slab. for state > 0, it's filled, and that's the index of the adsorbate atom in slab
state = np.zeros(len(coords), dtype=int)

temp = 300
pot = 2

slab calc <ase.calculators.eam.EAM object at 0x7f869f615490>
In pristine slab, there are a total of 64 sites


/home/dux/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/home/dux/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r2_topology = np.array(self.r2_topology)


In [3]:
Cu = catkit.gratoms.Gratoms('Cu')
atom = Cu
new_slab = slab.copy()

proposed_slab_builder = catkit.gen.adsorption.Builder(new_slab)

# add multiple adsorbates
site_indices = list(range(len(connectivity)))

print(f"site index is {site_indices[0]}")
# use proposed_slab_builder._single_adsorption multiple times
for i, index in enumerate(site_indices):
    new_slab = proposed_slab_builder._single_adsorption(
                atom,
                bond=0,
                slab=new_slab,
                site_index=site_indices[i],
                auto_construct=False,
                symmetric=False)

write('new_slab.cif', new_slab)

# store the actual positions of the sides
print(f"new slab has {len(new_slab)} atoms and original slab has {len(slab)} atoms.")
ads_coords = new_slab.get_positions()[len(slab):]

site index is 0
new slab has 128 atoms and original slab has 64 atoms.


In [11]:
assert len(ads_coords) == len(coords)
pp(connectivity)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])


In [12]:
# del slab[1]
# get a 4 connectivity (hollow)
slab.append('Cu')
slab.positions[-1] = ads_coords[-1]
# slab.get_positions()

# get a 2 connectivity (bridge)
slab.append('Cu')
slab.positions[-1] = ads_coords[16]

# get a 1 connecitivty (top)
slab.append('Cu')
slab.positions[-1] = ads_coords[0]

write('manual_append.cif', slab)

In [13]:
print(f"slab calc {slab.calc}")
# EAM calc retained!

slab calc <ase.calculators.eam.EAM object at 0x1086f8f40>


In [14]:
del slab[-1]
write('manual_append_del1.cif', slab)
del slab[-1]
del slab[-1]
write('manual_append_del3.cif', slab)